In [ ]:
#Note: donor vector pDO23A has a point mutation in our stock, but it is not located in a crucial site.
#The point mutation is at index 124 (T-->C) in the following part of the backbone:
#reverse_complement_before_insert = "ACTTTGTACAAGAAAGTTGAACGAGAAACGTAAAATGATATAAATATCAATATATTAAATTAGATTTTGCATAAAAAACAGACTACATAATACTGTAAAACACAACATATGCAGTCACTATGAACCAACTACTTAGATGGTATTAGTGACCTGTAGAATTCGAGCTCTAGAGCTGCAGGGCCCGAGCTTAAGACTGGCCGTCGTTTTACAACGTCGTGACTGGGAAAACATCCATGCTAGCGTTAACGCGAGAGTAGGGAACTGCCAGGCATCAAATAAAACGAAAGGCTCAGTCGGAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCCTGAGTAGGACAAATCCGCCGGGAGCGGATTTGAACGTTGTGAAGCAACGGCCCGGAGGGTGGCGGGCAGGACGCCCGCCATAAACTGCCAGGCATCAAACTAAGCAGAAGGCCATCCTGACGGATGGCCTTTTTGCGTTTCTACAAACTCTTCCTGGCTAGCGGTACGCGTATTAATTGCGTTGCGCTCACTGCCCGCTTTCCAGTCGGGAAACCTGTCGTGCCAGCTGCATTAATGAATCGGCCAACGCGCGGGGAGAGGCGGTTTGCGTATTGGGCGCTCTTCCGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTAT"

In [1]:
!pip install Biopython
from Bio.PDB import *
from Bio.Seq import Seq
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.3 MB/s eta 0:00:00


In [168]:
designs = pd.read_excel("Design wet lab validation master spreadsheet.xlsx")
sequencing_results = pd.read_csv("1101309_summary.csv")

In [160]:
# #optional- see what nucleotides flank the gene sequence
# first_att_list = []
# second_att_list = []

In [169]:
def check_sequence(sequence, BP_insert):
  before_insert = "ATAAGGCGCAGCGGTCGGGCTGAACGGGGGGTTCGTGCACACAGCCCAGCTTGGAGCGAACGACCTACACCGAACTGAGATACCTACAGCGTGAGCTATGAGAAAGCGCCACGCTTCCCGAAGGGAGAAAGGCGGACAGGTATCCGGTAAGCGGCAGGGTCGGAACAGGAGAGCGCACGAGGGAGCTTCCAGGGGGAAACGCCTGGTATCTTTATAGTCCTGTCGGGTTTCGCCACCTCTGACTTGAGCGTCGATTTTTGTGATGCTCGTCAGGGGGGCGGAGCCTATGGAAAAACGCCAGCAACGCGGCCTTTTTACGGTTCCTGGCCTTTTGCTGGCCTTTTGCTCACATGTTCTTTCCTGCGTTATCCCCTGATTCTGTGGATAACCGTATTACCGCCTTTGAGTGAGCTGATACCGCTCGCCGCAGCCGAACGACCGAGCGCAGCGAGTCAGTGAGCGAGGAAGCGGAAGAGCGCCCAATACGCAAACCGCCTCTCCCCGCGCGTTGGCCGATTCATTAATGCAGCTGGCACGACAGGTTTCCCGACTGGAAAGCGGGCAGTGAGCGCAACGCAATTAATACGCGTACCGCTAGCCAGGAAGAGTTTGTAGAAACGCAAAAAGGCCATCCGTCAGGATGGCCTTCTGCTTAGTTTGATGCCTGGCAGTTTATGGCGGGCGTCCTGCCCGCCACCCTCCGGGCCGTTGCTTCACAACGTTCAAATCCGCTCCCGGCGGATTTGTCCTACTCAGGAGAGCGTTCACCGACAAACAACAGATAAAACGAAAGGCCCAGTCTTCCGACTGAGCCTTTCGTTTTATTTGATGCCTGGCAGTTCCCTACTCTCGCGTTAACGCTAGCATGGATGTTTTCCCAGTCACGACGTTGTAAAACGACGGCCAGTCTTAAGCTCGGGCCCTGCAGCTCTAGAGCTCGAATTCTACAGGTCACTAATACCATCTAAGTAGTTGGTTCATAGTGACTGCATATGTTGTGTTTTACAGTATTATGTAGTCTGTTTTTTATGCAAAATCTAATTTAATATATTGATATTTATATCATTTTACGTTTCTCGTTCAACTTTCTTGTACAAAGT"
  after_insert = "CAACTTTATTATACAAAGTTGGCATTATAAAAAAGCATTGCTTATCAATTTGTTGCAACGAACAGGTCACTATCAGTCAAAATAAAATCATTATTTGGAGCTCCATGGTAGCGTTAACGCGGCCGCGATGTCAGGTGGCACTTTTCGGGGAAATGTGCGCGGAACCCCTATTTGTTTATTTTTCTAAATACATTCAAATATGTATCCGCTCATGAGACAATAACCCTGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGGCATTTTGCCTTCCTGTTTTTGCTCACCCAGAAACGCTGGTGAAAGTAAAAGATGCTGAAGATCAGTTGGGTGCACGAGTGGGTTACATCGAACTGGATCTCAACAGCGGTAAGATCCTTGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTAAAGTTCTGCTATGTGGCGCGGTATTATCCCGTATTGACGCCGGGCAAGAGCAACTCGGTCGCCGCATACACTATTCTCAGAATGACTTGGTTGAGTACTCACCAGTCACAGAAAAGCATCTTACGGATGGCATGACAGTAAGAGAATTATGCAGTGCTGCCATAACCATGAGTGATAACACTGCGGCCAACTTACTTCTGACAACGATCGGAGGACCGAAGGAGCTAACCGCTTTTTTGCACAACATGGGGGATCATGTAACTCGCCTTGATCGTTGGGAACCGGAGCTGAATGAAGCCATACCAAACGACGAGCGTGACACCACGATGCCTGTAGCAATGGCAACAACGTTGCGCAAACTATTAACTGGCGAACTACTTACTCTAGCTTCCCGGCAACAATTAATAGACTGGATGGAGGCGGATAAAGTTGCAGGACCACTTCTGCGCTCGGCCCTTCCGGCTGGCTGGTTTATTGCTGATAAATCTGGAGCCGGTGAGCGTGGGTCTCGCGGTATCATTGCAGCACTGGGGCCAGATGGTAAGCCCTCCCGTATCGTAGTTATCTACACGACGGGGAGTCAGGCAACTATGGATGAACGAAATAGACAGATCGCTGAGATAGGTGCCTCACTGATTAAGCATTGGTAACTGTCAGACCAAGTTTACTCATATATACTTTAGATTGATTTAAAACTTCATTTTTAATTTAAAAGGATCTAGGTGAAGATCCTTTTTGATAATCTCATGACCAAAATCCCTTAACGTGAGTTTTCGTTCCACTGAGCGTCAGACCCCGTAGAAAAGATCAAAGGATCTTCTTGAGATCCTTTTTTTCTGCGCGTAATCTGCTGCTTGCAAACAAAAAAACCACCGCTACCAGCGGTGGTTTGTTTGCCGGATCAAGAGCTACCAACTCTTTTTCCGAAGGTAACTGGCTTCAGCAGAGCGCAGATACCAAATACTGTTCTTCTAGTGTAGCCGTAGTTAGGCCACCACTTCAAGAACTCTGTAGCACCGCCTACATACCTCGCTCTGCTAATCCTGTTACCAGTGGCTGCTGCCAGTGGCGATAAGTCGTGTCTTACCGGGTTGGACTCAAGACGATAGTTACCGG"
  start_attb = "GGGGACAGCTTTCTTGTACAAAGTGGT"
  end_attb = "TACAACTTTATTATACAAAGTTGTCCCC"
  gene = BP_insert.replace(start_attb,"")
  gene = gene.replace(end_attb,"")

  result = ""
  if sequence[0:1100] != before_insert or sequence[-1589:] != after_insert:
    result += " Backbone segments altered around insert."
  if gene not in sequence[1100:-1589]:
    result += " Insert sequence is incorrect or missing."
  elif sequence == before_insert + "GGT" + gene + "TA" + after_insert:
    result = " Correct."
  else:
    result = " Unknown problem in sequence."

#This commented version was used before observing that "GGT" and "TA" will always flank the gene post-recombination
  # elif sequence[0:1100] == before_insert and sequence[-1589:] == after_insert and gene in sequence[1100:-1589]:
  #   result = " Correct."
  #   gene_index = sequence.find(gene)
  #   first_att = sequence[1100:gene_index]
  #   second_att = sequence[gene_index + len(gene):-1589]
  #   first_att_list.append(first_att)
  #   second_att_list.append(second_att)

  return result


In [170]:
for index in sequencing_results["Sample Name"].index:
  sample = sequencing_results.loc[index,"Sample Name"]
  sequence = sequencing_results.loc[index,"Consensus Sequence"]
  for index_designs in designs["BP colonies sequenced"].index:
    colonies = ""
    if type(designs.loc[index_designs, "BP colonies sequenced"]) == str:
      colonies = designs.loc[index_designs, "BP colonies sequenced"].split(";")
    if sample in colonies:
      if sequencing_results.loc[index,"Sample Status"] == "Fail":
        if pd.isnull(designs.loc[index_designs, "BP sequencing results"]) == True:
          designs.loc[index_designs, "BP sequencing results"] = sample + ":" + " Sequencing failed."
        elif type(designs.loc[index_designs, "BP sequencing results"]) == str:
          all_results = designs.loc[index_designs, "BP sequencing results"]
          all_results += " " + sample + ":" + " Sequencing failed."
          designs.loc[index_designs, "BP sequencing results"] = all_results
        continue

      BP_insert = designs.loc[index_designs, "BP cloning sequence"]
      result = check_sequence(sequence, BP_insert)
      if pd.isnull(designs.loc[index_designs, "BP sequencing results"]) == True:
        designs.loc[index_designs, "BP sequencing results"] = sample + ":" + result
        # print(designs.loc[index_designs, "BP sequencing results"])
      elif type(designs.loc[index_designs, "BP sequencing results"]) == str:
        all_results = designs.loc[index_designs, "BP sequencing results"]
        all_results += " " + sample + ":" + result
        designs.loc[index_designs, "BP sequencing results"] = all_results
        # print(all_results)


In [171]:
designs.to_excel("Designs_sequencing_results.xlsx")